# Snap to Road - Ajustar puntos a la red vial

Este notebook ajusta las coordenadas de rejas/puntos para que queden exactamente sobre las calles más cercanas usando datos de OpenStreetMap.

**Autor:** Proyecto Rejas La Florida  
**Fecha:** Diciembre 2025

## 1. Instalar dependencias

Ejecuta esta celda solo la primera vez:

In [ ]:
# Instalar librerias necesarias (solo ejecutar una vez)
!pip install pandas openpyxl osmnx scipy numpy

## 2. Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import osmnx as ox
from scipy.spatial import cKDTree

print("Librerias cargadas correctamente!")

## 3. Configuración

**Modifica estas variables según tu archivo:**

In [ ]:
# ========================================
# CONFIGURACION - MODIFICAR AQUI
# ========================================

ARCHIVO_ENTRADA = "tu_archivo.xlsx"  # Cambia por tu archivo
ARCHIVO_SALIDA = "tu_archivo_snapped.xlsx"  # Nombre del archivo de salida
LUGAR = "La Florida, Santiago, Chile"  # Lugar para descargar red vial

# ========================================

## 4. Cargar datos

El archivo debe tener columnas `lat` y `lon`, o una columna `cord` con formato "lat, lon"

In [ ]:
# Cargar archivo Excel
df = pd.read_excel(ARCHIVO_ENTRADA)

print(f"Archivo cargado: {len(df)} filas")
print(f"Columnas: {list(df.columns)}")
df.head()

In [ ]:
# Si no tienes columnas lat/lon, extraerlas de 'cord' o 'Coordenadas'

def extraer_coordenadas(cord_str):
    """Extrae lat, lon de un string con formato 'lat, lon'"""
    try:
        parts = str(cord_str).split(',')
        lat = float(parts[0].strip())
        lon = float(parts[1].strip())
        return lat, lon
    except:
        return None, None

# Detectar columna de coordenadas
if 'lat' not in df.columns or 'lon' not in df.columns:
    if 'cord' in df.columns:
        print("Extrayendo lat/lon de columna 'cord'...")
        coords = df['cord'].apply(extraer_coordenadas)
        df['lat'] = coords.apply(lambda x: x[0])
        df['lon'] = coords.apply(lambda x: x[1])
    elif 'Coordenadas' in df.columns:
        print("Extrayendo lat/lon de columna 'Coordenadas'...")
        coords = df['Coordenadas'].apply(extraer_coordenadas)
        df['lat'] = coords.apply(lambda x: x[0])
        df['lon'] = coords.apply(lambda x: x[1])
    else:
        print("ERROR: No se encontraron columnas de coordenadas")
else:
    print("Columnas lat/lon encontradas")

# Eliminar filas sin coordenadas
df = df.dropna(subset=['lat', 'lon'])
print(f"\nPuntos con coordenadas validas: {len(df)}")

## 5. Descargar red vial de OpenStreetMap

Esto puede tomar 1-2 minutos la primera vez:

In [ ]:
print(f"Descargando red vial de: {LUGAR}")
print("Esto puede tomar 1-2 minutos...")

try:
    G = ox.graph_from_place(LUGAR, network_type='drive', simplify=True)
except:
    print("No se pudo descargar por nombre, usando area centrada en los datos...")
    centro_lat = df['lat'].mean()
    centro_lon = df['lon'].mean()
    G = ox.graph_from_point((centro_lat, centro_lon), dist=5000, 
                             network_type='drive', simplify=True)

print(f"\nRed descargada exitosamente!")
print(f"  - Nodos (intersecciones): {len(G.nodes)}")
print(f"  - Aristas (calles): {len(G.edges)}")

## 6. Ajustar puntos a las calles

Para cada punto, buscamos el nodo más cercano en la red vial:

In [ ]:
# Preparar estructura de busqueda rapida
print("Preparando algoritmo de busqueda...")

nodes = list(G.nodes(data=True))
node_coords = np.array([[data['y'], data['x']] for _, data in nodes])
tree = cKDTree(node_coords)

print("Listo!")

In [ ]:
# Ajustar cada punto al nodo mas cercano
print(f"Ajustando {len(df)} puntos a la red vial...")

new_lats = []
new_lons = []
distances = []

for idx, row in df.iterrows():
    # Buscar nodo mas cercano
    dist, i = tree.query([row['lat'], row['lon']])
    
    # Guardar nueva posicion
    new_lats.append(node_coords[i][0])
    new_lons.append(node_coords[i][1])
    
    # Calcular distancia en metros
    dist_metros = dist * 111000  # 1 grado ~ 111km
    distances.append(dist_metros)
    
    # Mostrar progreso cada 500 puntos
    if (idx + 1) % 500 == 0:
        print(f"  {idx + 1}/{len(df)} procesados...")

print("\nAjuste completado!")

In [ ]:
# Agregar columnas al dataframe
df['lat_original'] = df['lat']
df['lon_original'] = df['lon']
df['lat'] = new_lats
df['lon'] = new_lons
df['dist_ajuste_m'] = distances

print("Columnas agregadas:")
print("  - lat, lon: Coordenadas ajustadas a la calle")
print("  - lat_original, lon_original: Coordenadas originales")
print("  - dist_ajuste_m: Distancia del ajuste en metros")

## 7. Resumen de resultados

In [ ]:
print("="*50)
print("RESUMEN DEL AJUSTE")
print("="*50)
print(f"Puntos procesados:   {len(df)}")
print(f"Ajuste promedio:     {np.mean(distances):.1f} metros")
print(f"Ajuste minimo:       {np.min(distances):.1f} metros")
print(f"Ajuste maximo:       {np.max(distances):.1f} metros")
print(f"Puntos con >50m:     {sum(1 for d in distances if d > 50)}")
print("="*50)

In [ ]:
# Ver primeras filas del resultado
df.head()

## 8. Guardar archivo

In [ ]:
# Guardar resultado
df.to_excel(ARCHIVO_SALIDA, index=False)

print(f"Archivo guardado: {ARCHIVO_SALIDA}")
print("\nListo! Ahora puedes usar este archivo para generar mapas.")

---

## Notas

- El ajuste mueve cada punto al **nodo de calle más cercano** en OpenStreetMap
- Puntos con ajuste mayor a 50 metros pueden estar mal ubicados originalmente
- La columna `dist_ajuste_m` te permite revisar qué puntos se movieron más